## EDA Lab

## Data understanding and preprocessing - Walkthrough of Traveler dataset




### * Goal: Predict the country that users will make their first booking in, based on some basic user profile data.




### Part1 - DATA PREPROCESSING (Data Cleaning and Data Transformation on train and test csv)

#### [1] Data Cleaning

In [1]:
##Exploring Traveler data
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline 

print("Reading data...")
train_file = "train_users_2.csv"
df_train = pd.read_csv(train_file, header = 0,index_col=None)

test_file = "test_users.csv"
df_test = pd.read_csv(test_file, header = 0,index_col=None)

# Combining into one dataset for cleaning
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True, sort=False)
print("Reading data...completed")

# Fixing date formats in Pandas - to_datetime
## Change dates to specific format
print("Fixing timestamps...")
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')
print("Fixing timestamps...completed")

## Removing date_first_booking column
df_all.drop('date_first_booking', axis = 1, inplace = True)
print("Droped date_first_booking column...")

import numpy as np

## Remove outliers function - [1]
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df

## Fixing age column - [2]
print("Fixing age column...")
df_all = remove_outliers(df = df_all, column = 'age', min_val = 15, max_val = 90)
df_all['age'].fillna(-1, inplace = True)
print("Fixing age column...completed")

# Other column missing value - Fill first_affiliate_tracked column
print("Filling first_affiliate_tracked column...")
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)
print("Filling first_affiliate_tracked column...completed")

df_all.head()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Reading data...
Reading data...completed
Fixing timestamps...
Fixing timestamps...completed
Droped date_first_booking column...
Fixing age column...
Fixing age column...completed
Filling first_affiliate_tracked column...
Filling first_affiliate_tracked column...completed


,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,-1.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


#### [2] Data Transformation and Feature Extraction

In [2]:
# Own implementation of One Hot Encoding - Data Transformation
def convert_to_binary(df, column_to_convert):
    categories = list(df[column_to_convert].drop_duplicates())

    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert[:5] + '_' + cat_name[:10]
        df[col_name] = 0
        df.loc[(df[column_to_convert] == category), col_name] = 1

    return df

# One Hot Encoding
print("One Hot Encoding categorical data...")
columns_to_convert = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

for column in columns_to_convert:
    df_all = convert_to_binary(df=df_all, column_to_convert=column)
    df_all.drop(column, axis=1, inplace=True)
print("One Hot Encoding categorical data...completed")

# Add new date related fields - Creating New Features
print("Adding new fields...")
df_all['day_account_created'] = df_all['date_account_created'].dt.weekday
df_all['month_account_created'] = df_all['date_account_created'].dt.month
df_all['quarter_account_created'] = df_all['date_account_created'].dt.quarter
df_all['year_account_created'] = df_all['date_account_created'].dt.year
df_all['hour_first_active'] = df_all['timestamp_first_active'].dt.hour
df_all['day_first_active'] = df_all['timestamp_first_active'].dt.weekday
df_all['month_first_active'] = df_all['timestamp_first_active'].dt.month
df_all['quarter_first_active'] = df_all['timestamp_first_active'].dt.quarter
df_all['year_first_active'] = df_all['timestamp_first_active'].dt.year
df_all['created_less_active'] = (df_all['date_account_created'] - df_all['timestamp_first_active']).dt.days
print("Adding new fields...completed")


# Drop unnecessary columns
print("Droping fields...")
columns_to_drop = ['date_account_created', 'timestamp_first_active', 'date_first_booking']
for column in columns_to_drop:
    if column in df_all.columns:
        df_all.drop(column, axis=1, inplace=True)
print("Droping fields...completed")

One Hot Encoding categorical data...
One Hot Encoding categorical data...completed
Adding new fields...
Adding new fields...completed
Droping fields...
Droping fields...completed


### Part2 - DATA PREPROCESSING (Data Cleaning and Data Transformation on sessions csv)


#### [1] Extract the primary and secondary devices for each user

In [3]:
## Loading sessions.csv data
print("Reading sessions data...")
sessions_file = "sessions.csv"
df_sessions = pd.read_csv(sessions_file, header = 0,index_col=False)
print("Reading sessions data...completed")
df_sessions.head()

print("Determing primary device...")
# Selection of all rows for 'user_id', 'device_type', 'secs_elapsed' using .loc operation
sessions_device = df_sessions.loc[:, ['user_id', 'device_type', 'secs_elapsed']]
#sessions_device.head(10)
# Grouping based on 'user_id', 'device_type' the sum of 'secs_elapsed' 
aggregated_lvl1 = sessions_device.groupby(['user_id', 'device_type'], as_index=False, sort=False).aggregate(np.sum)
#aggregated_lvl1.head(10)
# Obtaining the index which is true or false based on first largest device type used by users as matching condition
idx = aggregated_lvl1.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == aggregated_lvl1['secs_elapsed']
#idx.head(10)
# Obtaining the rows of first largest device type used by users based on index
df_sessions_primary = pd.DataFrame(aggregated_lvl1.loc[idx, ['user_id', 'device_type', 'secs_elapsed']])
#df_sessions_primary.head(10)
# Rename the attributes and modify in the df_sessions_primary dataframe
df_sessions_primary.rename(columns = {'device_type':'primary_device','secs_elapsed':'primary_secs'}, inplace=True)
#df_sessions_primary.head(10)
# Call user defined One Hot Encoding function
df_sessions_primary = convert_to_binary(df=df_sessions_primary, column_to_convert='primary_device')
#df_sessions_primary.head(10)
# drop the 'primary_device' attribute after one-hot encoding
df_sessions_primary.drop('primary_device', axis=1, inplace=True)
#df_sessions_primary.head(10)
print("Determing primary device...completed")
print("Determing secondary device...")
# Initially drop the primary device index before selecting the remaining device (eg: secondary device)
remaining = aggregated_lvl1.drop(aggregated_lvl1.index[idx])
#remaining.head(10)
# Obtaining the index which is true or false based on second largest device type used by users as matching condition
idx = remaining.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == remaining['secs_elapsed']
#idx.head(10)
# Obtaining the rows of second largest device type used by users based on index
df_sessions_secondary = pd.DataFrame(remaining.loc[idx , ['user_id', 'device_type', 'secs_elapsed']])
#df_sessions_secondary.head(10)
# Rename the attributes and modify in the df_sessions_secondary dataframe
df_sessions_secondary.rename(columns = {'device_type':'secondary_device', 'secs_elapsed':'secondary_secs'}, inplace=True)
#df_sessions_secondary.head(10)
# Call user defined One Hot Encoding function
df_sessions_secondary = convert_to_binary(df=df_sessions_secondary, column_to_convert='secondary_device')
#df_sessions_secondary.head(10)
# drop the 'secondary_device' attribute after one-hot encoding
df_sessions_secondary.drop('secondary_device', axis=1, inplace=True)
#df_sessions_secondary.head(10)
print("Determing secondary device...completed")
## Testing ...
#df_sessions_secondary.head()


Reading sessions data...
Reading sessions data...completed
Determing primary device...
Determing primary device...completed
Determing secondary device...
Determing secondary device...completed


#### [2] Determine Counts of Actions attributes

In [4]:
# Count occurrences of value in a column
def convert_to_counts(df, id_col, column_to_convert):
    #id_list = df[id_col].drop_duplicates()
    #print (id_list.head())
    #Step1
    df_counts = df.loc[:,[id_col, column_to_convert]]  
    df_counts['count'] = 1
    df_counts = df_counts.groupby(by=[id_col, column_to_convert], as_index=False, sort=False).sum()
    print('Step1')
    print (df_counts.head())
    #Step2
    new_df = df_counts.pivot(index=id_col, columns=column_to_convert, values='count') 
    new_df = new_df.fillna(0)
    print ('Step2')
    print (new_df.head())

    # Rename Columns
    categories = list(df[column_to_convert].drop_duplicates())
    print('categories')
    print (categories)
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert + '_' + cat_name
        new_df.rename(columns = {category:col_name}, inplace=True)
        
    return new_df

# Aggregate and combine actions taken columns
print("Aggregating actions taken...")
session_actions = df_sessions.loc[:,['user_id', 'action', 'action_type', 'action_detail']]
#session_actions = df_sessions.loc[:,['user_id', 'action']]
#session_actions.head()
#columns_to_convert = ['action']
columns_to_convert = ['action', 'action_type', 'action_detail']
session_actions = session_actions.fillna('not provided')
#session_actions.head()
first = True
for column in columns_to_convert:
    print("Converting " + column + " attribute...")
    current_data = convert_to_counts(df=session_actions, id_col='user_id', column_to_convert=column)
    print("Converting " + column + " attribute... finished")
# If first loop, current data becomes existing data, otherwise merge existing and current
    if first:
        first = False
        actions_data = current_data
    else:
        actions_data = pd.concat([actions_data, current_data], axis=1, join='inner')
    


Aggregating actions taken...
Converting action attribute...
Step1
      user_id            action  count
0  d1mm9tcy42            lookup     23
1  d1mm9tcy42    search_results     12
2  d1mm9tcy42       personalize     25
3  d1mm9tcy42             index     11
4  d1mm9tcy42  similar_listings      9
Step2
action       10   11   12   15  about_us  accept_decline  account  \
user_id                                                             
00023iyk9l  0.0  0.0  0.0  0.0       0.0             0.0      0.0   
0010k6l0om  0.0  0.0  0.0  0.0       0.0             0.0      0.0   
001wyh0pz8  0.0  0.0  0.0  0.0       0.0             0.0      0.0   
0028jgx1x1  0.0  0.0  0.0  0.0       0.0             0.0      0.0   
002qnbzfs5  9.0  0.0  0.0  0.0       0.0             0.0      0.0   

action      acculynk_bin_check_failed  acculynk_bin_check_success  \
user_id                                                             
00023iyk9l                        0.0                         0.0   
001

Converting action attribute... finished
Converting action_type attribute...
Step1
      user_id   action_type  count
0  d1mm9tcy42  not provided     37
1  d1mm9tcy42         click     20
2  d1mm9tcy42          data     35
3  d1mm9tcy42          view     25
4  d1mm9tcy42        submit      1
Step2
action_type  -unknown-  booking_request  booking_response  click   data  \
user_id                                                                   
00023iyk9l         0.0              1.0               0.0    4.0    9.0   
0010k6l0om         5.0              0.0               0.0   16.0    9.0   
001wyh0pz8         6.0              0.0               0.0   66.0    2.0   
0028jgx1x1         1.0              0.0               0.0    9.0    5.0   
002qnbzfs5       184.0              1.0               0.0  140.0  140.0   

action_type  message_post  modify  not provided  partner_callback  submit  \
user_id                                                                     
00023iyk9l            

In [5]:
##Checking...
actions_data.head()

,action_10,action_11,action_12,action_15,action_about_us,action_accept_decline,action_account,action_acculynk_bin_check_failed,action_acculynk_bin_check_success,action_acculynk_load_pin_pad,...,action_detail_view_resolutions,action_detail_view_search_results,action_detail_view_security_checks,action_detail_view_user_real_names,action_detail_wishlist,action_detail_wishlist_content_update,action_detail_wishlist_note,action_detail_your_listings,action_detail_your_reservations,action_detail_your_trips
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0
0010k6l0om,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
001wyh0pz8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028jgx1x1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002qnbzfs5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### [3] Data Integration (combine datasets)

In [6]:
# [4.1] Merge device datasets
print("Combining df_primary and df_secondary to device dataframe...")
df_sessions_primary.set_index('user_id', inplace=True)
df_sessions_secondary.set_index('user_id', inplace=True)
device_data = pd.concat([df_sessions_primary, df_sessions_secondary], axis=1, join="outer", sort=False)
print("Combining df_primary and df_secondary to device dataframe...finished")

# [4.2] Merge device and actions datasets
print("Combining device and actions to sessions dataframe...")
combined_results = pd.concat([device_data, actions_data], axis=1, join='outer', sort=False)
df_sessions_complete = combined_results.fillna(0)
print("Combining device and actions to sessions dataframe...finished")

# [4.3] Merge user and session datasets
print("Combining sessions and users to get final dataframe...")
df_all.set_index('id', inplace=True)
df_all = pd.concat([df_all, df_sessions_complete], axis=1, join='inner', sort = False)
print("Combining sessions and users to get final dataframe...finished")



Combining df_primary and df_secondary to device dataframe...
Combining df_primary and df_secondary to device dataframe...finished
Combining device and actions to sessions dataframe...
Combining device and actions to sessions dataframe...finished
Combining sessions and users to get final dataframe...
Combining sessions and users to get final dataframe...finished


In [7]:
df_all.head()

,age,country_destination,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,signu_basic,signu_google,signu_weibo,...,action_detail_view_resolutions,action_detail_view_search_results,action_detail_view_security_checks,action_detail_view_user_real_names,action_detail_wishlist,action_detail_wishlist_content_update,action_detail_wishlist_note,action_detail_your_listings,action_detail_your_reservations,action_detail_your_trips
d1mm9tcy42,62.0,other,0,1,0,0,0,1,0,0,...,0.0,23.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
yo8nz8bqcq,-1.0,NDF,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4grx6yxeby,-1.0,NDF,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ncf87guaf0,-1.0,NDF,1,0,0,0,0,1,0,0,...,0.0,32.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
4rvqpxoh3h,-1.0,GB,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### End of Preprocessing Part2

In [8]:
df_all.to_csv("td.csv",index=False)